<a href="https://colab.research.google.com/github/katiatel/ml/blob/main/exam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import DistanceMetric

In [ ]:
d = pd.read_csv('pulsar_stars_new.csv', decimal='.', delimiter=",")

ps = d[((d.TARGET == 0) & (d.MIP >= 83) & (d.MIP <= 84)) |
          ((d.TARGET == 1) & (d.MIP >= 83) & (d.MIP <= 89))]

print("1: ", len(ps))
print("2: ", ps.MIP.mean())

1:  136
2:  84.54279641544117


In [ ]:
scaler = MinMaxScaler()
scaler.fit(ps)
ps = pd.DataFrame(scaler.transform(ps), columns=ps.columns)

np.mean(ps['MIP'])

0.26595638255302173

In [ ]:
st = [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1] #меняем числа


In [ ]:
X = pd.DataFrame(ps.drop(["TARGET"], axis=1))
y = pd.DataFrame(ps["TARGET"])

In [ ]:
reg = LogisticRegression(random_state=2019, solver='lbfgs').fit(X, y.values.ravel()) #параметры вписать
for i, p in enumerate(reg.predict_proba([st])[0]):
    print(("не пульсар" if i == 0 else "пульсар") + "\": ", p) #внимание пульсар или непульсар

не пульсар":  0.7264450276675015
пульсар":  0.2735549723324985


In [ ]:
for m in ["euclidean", "manhattan"]: #выбираем нужную метрику если не указано ничего не меняем
    dist = DistanceMetric.get_metric(m)
    dist1 = [dist.pairwise(np.concatenate(([i], [st])))[0][1] for i in list(X.values)]

In [ ]:
    print(m + ":", min(dist1))

manhattan: 0.5478999978926339


задание 2

In [ ]:
!pip install mnist

In [ ]:
import pandas as pd
import numpy as np
import mnist
import warnings

In [ ]:
from keras.datasets import mnist
from sklearn.decomposition import PCA
exp_disp = 0.77

(X_tr, y_tr), (X_pr, y_pr) = mnist.load_data()
dim = 784
X_tr_ = X_tr.reshape(len(X_tr), dim)

pca = PCA(svd_solver='full')
pca = pca.fit(X_tr_)

M = 0
for arg, val in enumerate(np.cumsum(pca.explained_variance_ratio_)):
    if val > exp_disp:
        M = arg + 1
        break

print(str(exp_disp) + ": " + str(M))

0.77: 37


In [ ]:
from sklearn.model_selection import train_test_split   
X_tr = X_tr.reshape(len(X_tr), dim)

pca = PCA(n_components=M, svd_solver='full')
pca = pca.fit(X_tr)

X_tr, X_test, y_tr, y_test = train_test_split(X_tr, y_tr, test_size=0.3, random_state=48)
    
X_tr = pca.transform(X_tr)
X_test = pca.transform(X_test)

print(str(X_tr.transpose()[0].mean()))

2.032837033185744


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import confusion_matrix

rf = RandomForestClassifier(criterion='gini', min_samples_leaf=10, max_depth=20, n_estimators=10, random_state=48)
clf_r_f = OneVsRestClassifier(rf).fit(X_tr, y_tr)
y_pr = clf_r_f.predict(X_test)

a = 5
con_m = confusion_matrix(y_test, y_pr)
print(str(con_m[a][a]))

1441


In [ ]:
from sklearn.linear_model import LogisticRegression

log_r = LogisticRegression(random_state=48, solver='lbfgs')
clf = OneVsRestClassifier(log_r).fit(X_tr, y_tr)
y_pr = clf.predict(X_test)

b = 8
con_m = confusion_matrix(y_test, y_pr)
print(str(con_m[b][b]))

1481


In [ ]:
from sklearn.tree import DecisionTreeClassifier

d_t_c = DecisionTreeClassifier(criterion='gini', min_samples_leaf=10, max_depth=20, random_state=48)
clf = OneVsRestClassifier(d_t_c).fit(X_tr, y_tr)
y_pr = clf.predict(X_test)

c = 6
con_m = confusion_matrix(y_test, y_pr)
print(str(con_m[c][c]))

1643
